In [1]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint

def run_NASEM():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    user_diet, animal_input, equation_selection = nd.read_csv_input('./input.csv')
        
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')
    feed_data.reset_index(inplace=True)
    feed_data = feed_data.rename(columns={"Fd_Name": "Feedstuff"})

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()
    diet_info_initial.insert(1, 'Fd_DMInp', Fd_DMInp.reindex(diet_info_initial['Feedstuff']).values)
    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    diet_info_initial['Fd_DNDF48'] = 0  

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del(list_of_feeds, Fd_DMInp)

    # Create infusion_input
    if equation_selection['use_infusions'] == 0:
         # If no infusions then all values are set to 0
        no_infusion_input = {'Inf_Acet_g': 0,
                             'Inf_ADF_g': 0,
                             'Inf_Arg_g': 0,
                             'Inf_Ash_g': 0,
                             'Inf_Butr_g': 0,
                             'Inf_CP_g': 0,
                             'Inf_CPARum_CP': 0,
                             'Inf_CPBRum_CP': 0,
                             'Inf_CPCRum_CP': 0,
                             'Inf_dcFA': 0,
                             'Inf_dcRUP': 0,
                             'Inf_DM_g': 0,
                             'Inf_EE_g': 0,
                             'Inf_FA_g': 0,
                             'Inf_Glc_g': 0,
                             'Inf_His_g': 0,
                             'Inf_Ile_g': 0,
                             'Inf_KdCPB': 0,
                             'Inf_Leu_g': 0,
                             'Inf_Lys_g': 0,
                             'Inf_Met_g': 0,
                             'Inf_NDF_g': 0,
                             'Inf_NPNCP_g': 0,
                             'Inf_Phe_g': 0,
                             'Inf_Prop_g': 0,
                             'Inf_St_g': 0,
                             'Inf_Thr_g': 0,
                             'Inf_Trp_g': 0,
                             'Inf_ttdcSt': 0,
                             'Inf_Val_g': 0,
                             'Inf_VFA_g': 0,
                             'Inf_Location': 0
                             }
    elif equation_selection['use_infusions'] == 1:
        infusion_input = nd.read_infusion_input('./infusion_input.csv')
    else:
        raise ValueError(f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")


########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8: 
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'], 
            animal_input['Trg_MilkProd'], 
            animal_input['An_BW'], 
            animal_input['An_BCS'],
            animal_input['An_LactDay'], 
            animal_input['Trg_MilkFatp'], 
            animal_input['Trg_MilkTPp'], 
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers    
    elif equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'], 
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF, 
            animal_input['An_BW'], 
            animal_input['An_BW_mature'], 
            animal_input['An_PrePartWk'], 
            nd.coeff_dict)

    
    elif equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'], 
            animal_input['An_BW'], 
            animal_input['An_PrePartWk'], 
            animal_input['An_GestDay'], 
            animal_input['An_GestLength'], 
            Dt_NDF, 
            nd.coeff_dict)
        
    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: "+ str(equation_selection['DMIn_eqn']))

    del(Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=nd.coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions
    
    diet_data = nd.calculate_diet_data(diet_info, 
                                       animal_input['DMI'], 
                                       animal_input['An_BW'],
                                       nd.coeff_dict)
    # diet_data contains everything starting with "Dt_"

########################################
# Step 4: Infusion Calculations
########################################
    if equation_selection['use_infusions'] == 0:
        infusion_data = nd.calculate_infusion_data(no_infusion_input,
                                                   animal_input['DMI'],
                                                   nd.coeff_dict)
    elif equation_selection['use_infusions'] == 1:
        infusion_data = nd.calculate_infusion_data(infusion_input,
                                                   animal_input['DMI'],
                                                   nd.coeff_dict)
    else:
        raise ValueError(f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")

########################################
# Step 5: Animal Level Calculations
########################################
# Combine Diet and Infusion nutrient supplies
    An_data = nd.calculate_An_data(animal_input,
                                   diet_data,
                                   infusion_data)
    
########################################
# Step 6: Rumen Digestion Calculations
########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data['Dt_NDFIn'],
                                       diet_data['Dt_StIn'],
                                       diet_data['Dt_CPIn'],
                                       diet_data['Dt_ADFIn'],
                                       diet_data['Dt_ForWet'])
                                       
    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data['Dt_ForNDF'],
                                     diet_data['Dt_StIn'],
                                     diet_data['Dt_ForWet'])
    
    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF, 
                                             diet_data['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data['Dt_StIn'])

########################################
# Step 7: Microbial Protein Calculations
########################################
    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                An_data['An_RDP'],
                                                An_data['An_RDPIn'])
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax, 
                                    nd.coeff_dict)
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_data['An_RDPIn_g'],
                                                 nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data['Dt_rOMIn'],
                                              diet_data['Dt_ForNDFIn'],
                                              An_data['An_RDPIn'],
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_data['An_RDPIn'],
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, nd.coeff_dict)

########################################
# Step 8: Amino Acid Calculations
########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values 

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g, 
                                                  AA_list, 
                                                  nd.coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'], 
                                                      nd.coeff_dict)
    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(diet_data, 
                                                  AA_values['Du_IdAAMic'], 
                                                  AA_list)
    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list, 
                                                    nd.coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'], 
                                         nd.coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'], 
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list, 
                                                    nd.coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'], 
                                                      AA_list, 
                                                      nd.coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'], 
                                                    AA_values['mPrt_AA_01'], 
                                                    AA_values['AA_mPrtmx'])
    
########################################
# Step 9: Other Calculations
########################################
    Uter_Wtpart = nd.calculate_Uter_Wtpart(animal_input['Fet_BWbrth'], nd.coeff_dict)
    Uter_Wt = nd.calculate_Uter_Wt(animal_input['An_Parity_rl'], 
                                animal_input['An_AgeDay'],
                                animal_input['An_LactDay'],
                                animal_input['An_GestDay'],
                                animal_input['An_GestLength'],
                                Uter_Wtpart,
                                nd.coeff_dict)
    GrUter_Wtpart = nd.calculate_GrUter_Wtpart(animal_input['Fet_BWbrth'], nd.coeff_dict)
    GrUter_Wt = nd.calculate_GrUter_Wt(animal_input['An_GestDay'],
                                    animal_input['An_GestLength'],
                                    Uter_Wt,
                                    GrUter_Wtpart,
                                    nd.coeff_dict)
    Uter_BWgain = nd.calculate_Uter_BWgain(animal_input['An_LactDay'], 
                                                    animal_input['An_GestDay'],
                                                    animal_input['An_GestLength'],
                                                    Uter_Wt,
                                                    nd.coeff_dict)
    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['An_LactDay'],
                                            animal_input['An_GestDay'],
                                            animal_input['An_GestLength'],
                                            GrUter_Wt,
                                            Uter_BWgain,
                                            nd.coeff_dict)

    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data

# ### RUN MODEL ###
# animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data = run_NASEM()


In [2]:
## RUN MODEL ###
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data = run_NASEM()

In [3]:
diet_info

,Feedstuff,Fd_DMInp,Fd_Libr,UID,Fd_Index,Fd_Category,Fd_Type,Fd_DM,Fd_Conc,Fd_Locked,...,Fd_IdArgRUPIn,Fd_IdHisRUPIn,Fd_IdIleRUPIn,Fd_IdLeuRUPIn,Fd_IdLysRUPIn,Fd_IdMetRUPIn,Fd_IdPheRUPIn,Fd_IdThrRUPIn,Fd_IdTrpRUPIn,Fd_IdValRUPIn
0,Alfalfa meal,0.334821,NRC 2020,NRC16F1,1,Plant Protein,Forage,90.749,0,1,...,3.512597,1.643258,3.486515,5.736041,3.845689,1.089536,3.827651,3.427432,1.414670,4.356124
1,Canola meal,0.274554,NRC 2020,NRC16F28,28,Plant Protein,Concentrate,89.128,100,1,...,4.235835,1.917361,2.964343,4.956856,3.950275,1.389536,2.855299,3.177254,0.958476,3.792306
2,"Corn silage, typical",0.223214,NRC 2020,NRC16F48,48,Grain Crop Forage,Forage,35.361,0,1,...,1.218378,0.907766,1.895612,4.506045,1.481940,0.824820,2.051839,1.805909,0.384554,2.464287
3,"Corn grain HM, coarse grind",0.167411,NRC 2020,NRC16F1072,256,Energy Source,Concentrate,72.267,100,1,...,3.015391,1.845747,2.261655,7.646549,1.913432,1.256475,3.094743,2.314233,0.475666,3.049495


# Test fragmented df

In [37]:

# Temporary function to test better way to iterate columns
from nasem_dairy.NASEM_equations.dev_nutrient_intakes import *
def DIcalculate_diet_info(DMI, An_StatePhys, Use_DNDF_IV, diet_info, coeff_dict, testmethod):
    
    # Start with copy of diet_info
    complete_diet_info = diet_info.copy()

    if testmethod == 'old':
        # Calculate all aditional feed data columns
        complete_diet_info['Fd_DMIn'] = calculate_Fd_DMIn(DMI, diet_info['Fd_DMInp'])
        complete_diet_info['Fd_GE'] = calculate_Fd_GE(An_StatePhys, 
                                                    diet_info['Fd_Category'],
                                                    diet_info['Fd_CP'],
                                                    diet_info['Fd_FA'],
                                                    diet_info['Fd_Ash'],
                                                    diet_info['Fd_St'],
                                                    diet_info['Fd_NDF'],
                                                    coeff_dict)
        complete_diet_info['Fd_AFIn'] = calculate_Fd_AFIn(diet_info['Fd_DM'], diet_info['Fd_DMIn'])
        complete_diet_info['Fd_For'] = calculate_Fd_For(diet_info['Fd_Conc'])
        complete_diet_info['Fd_ForWet'] = calculate_Fd_ForWet(diet_info['Fd_DM'], complete_diet_info['Fd_For'])
        complete_diet_info['Fd_ForDry'] = calculate_Fd_ForDry(diet_info['Fd_DM'], complete_diet_info['Fd_For'])
        complete_diet_info['Fd_Past'] = calculate_Fd_Past(diet_info['Fd_Category'])
        complete_diet_info['Fd_LiqClf'] = calculate_Fd_LiqClf(diet_info['Fd_Category'])
        complete_diet_info['Fd_ForNDF'] = calculate_Fd_ForNDF(diet_info['Fd_NDF'], diet_info['Fd_Conc'])
        complete_diet_info['Fd_NDFnf'] = calculate_Fd_NDFnf(diet_info['Fd_NDF'], diet_info['Fd_NDFIP'])
        complete_diet_info['Fd_NPNCP'] = calculate_Fd_NPNCP(diet_info['Fd_CP'], diet_info['Fd_NPN_CP'])
        complete_diet_info['Fd_NPN'] = calculate_Fd_NPN(complete_diet_info['Fd_NPNCP'])
        complete_diet_info['Fd_NPNDM'] = calculate_Fd_NPNDM(complete_diet_info['Fd_NPNCP'])
        complete_diet_info['Fd_TP'] = calculate_Fd_TP(diet_info['Fd_CP'], complete_diet_info['Fd_NPNCP'])
        complete_diet_info['Fd_fHydr_FA'] = calculate_Fd_fHydr_FA(diet_info['Fd_Category'])
        complete_diet_info['Fd_FAhydr'] = calculate_Fd_FAhydr(diet_info['Fd_FA'], complete_diet_info['Fd_fHydr_FA'])
        complete_diet_info['Fd_NFC'] = calculate_Fd_NFC(diet_info['Fd_NDF'], 
                                                        complete_diet_info['Fd_TP'], 
                                                        diet_info['Fd_Ash'], 
                                                        complete_diet_info['Fd_FAhydr'], 
                                                        complete_diet_info['Fd_NPNDM'])
        complete_diet_info['Fd_rOM'] = calculate_Fd_rOM(diet_info['Fd_NDF'], 
                                                        diet_info['Fd_St'], 
                                                        complete_diet_info['Fd_TP'], 
                                                        diet_info['Fd_FA'], 
                                                        complete_diet_info['Fd_fHydr_FA'], 
                                                        diet_info['Fd_Ash'], 
                                                        complete_diet_info['Fd_NPNDM'])
    
    if testmethod == 'new':
        complete_diet_info = complete_diet_info.assign(
            Fd_DMIn=lambda df: calculate_Fd_DMIn(DMI, diet_info['Fd_DMInp']),
            Fd_GE=lambda df: calculate_Fd_GE(An_StatePhys, diet_info['Fd_Category'], diet_info['Fd_CP'], diet_info['Fd_FA'],
                                            diet_info['Fd_Ash'], diet_info['Fd_St'], diet_info['Fd_NDF'], coeff_dict),
            Fd_AFIn=lambda df: calculate_Fd_AFIn(diet_info['Fd_DM'], df['Fd_DMIn']),
            Fd_For=lambda df: calculate_Fd_For(diet_info['Fd_Conc']),
            Fd_ForWet=lambda df: calculate_Fd_ForWet(diet_info['Fd_DM'], df['Fd_For']),
            Fd_ForDry=lambda df: calculate_Fd_ForDry(diet_info['Fd_DM'], df['Fd_For']),
            Fd_Past=lambda df: calculate_Fd_Past(diet_info['Fd_Category']),
            Fd_LiqClf=lambda df: calculate_Fd_LiqClf(diet_info['Fd_Category']),
            Fd_ForNDF=lambda df: calculate_Fd_ForNDF(diet_info['Fd_NDF'], diet_info['Fd_Conc']),
            Fd_NDFnf=lambda df: calculate_Fd_NDFnf(diet_info['Fd_NDF'], diet_info['Fd_NDFIP']),
            Fd_NPNCP=lambda df: calculate_Fd_NPNCP(diet_info['Fd_CP'], diet_info['Fd_NPN_CP']),
            Fd_NPN=lambda df: calculate_Fd_NPN(df['Fd_NPNCP']),
            Fd_NPNDM=lambda df: calculate_Fd_NPNDM(df['Fd_NPNCP']),
            Fd_TP=lambda df: calculate_Fd_TP(diet_info['Fd_CP'], df['Fd_NPNCP']),
            Fd_fHydr_FA=lambda df: calculate_Fd_fHydr_FA(diet_info['Fd_Category']),
            Fd_FAhydr=lambda df: calculate_Fd_FAhydr(diet_info['Fd_FA'], df['Fd_fHydr_FA']),
            Fd_NFC=lambda df: calculate_Fd_NFC(diet_info['Fd_NDF'], df['Fd_TP'],
                                            diet_info['Fd_Ash'], df['Fd_FAhydr'],
                                            df['Fd_NPNDM']),
            Fd_rOM=lambda df: calculate_Fd_rOM(diet_info['Fd_NDF'], diet_info['Fd_St'], df['Fd_TP'],
                                            diet_info['Fd_FA'], df['Fd_fHydr_FA'],
                                            diet_info['Fd_Ash'], df['Fd_NPNDM'])
        )



    # Loop through identical calculations
    column_names_XIn = ['Fd_ADF', 
                        'Fd_NDF', 
                        'Fd_St', 
                        'Fd_NFC', 
                        'Fd_WSC', 
                        'Fd_rOM', 
                        'Fd_Lg', 
                        'Fd_Conc', 
                        'Fd_For', 
                        'Fd_ForWet', 
                        'Fd_ForDry', 
                        'Fd_Past', 
                        'Fd_CP',
                        'Fd_TP',
                        'Fd_CFat',
                        'Fd_FA',
                        'Fd_FAhydr',
                        'Fd_Ash',
                        'Fd_GE'
                        ]
    
    if testmethod == 'old':
        for column_name in column_names_XIn:
            complete_diet_info[f"{column_name}In"] = complete_diet_info.apply(lambda row: row[column_name] / 100 * row['Fd_DMIn'], axis=1)

    if testmethod == 'new':
        complete_diet_info = complete_diet_info.assign(
            **{f"{col}In": lambda df: df[col] / 100 * df['Fd_DMIn'] for col in column_names_XIn}
        )

    if testmethod == 'old':
        # Calculate nutrient intakes for each feed
        complete_diet_info['TT_dcFdNDF_Lg'] = calculate_TT_dcFdNDF_Lg(diet_info['Fd_NDF'], 
                                                                    diet_info['Fd_Lg'])
        complete_diet_info['Fd_DNDF48'] = calculate_Fd_DNDF48(diet_info['Fd_Conc'],
                                                            diet_info['Fd_DNDF48'])
        complete_diet_info['TT_dcFdNDF_48h'] = calculate_TT_dcFdNDF_48h(complete_diet_info['Fd_DNDF48'])
        complete_diet_info['TT_dcFdNDF_Base'] = calculate_TT_dcFdNDF_Base(Use_DNDF_IV,
                                                                        diet_info['Fd_Conc'],
                                                                        complete_diet_info['TT_dcFdNDF_Lg'],
                                                                        complete_diet_info['TT_dcFdNDF_48h'])
        complete_diet_info['Fd_DigNDFIn_Base'] = calculate_Fd_DigNDFIn_Base(complete_diet_info['Fd_NDFIn'], 
                                                                            complete_diet_info['TT_dcFdNDF_Base'])
        complete_diet_info['Fd_NPNCPIn'] = calculate_Fd_NPNCPIn(complete_diet_info['Fd_CPIn'], complete_diet_info['Fd_NPN_CP'])
        complete_diet_info['Fd_NPNIn'] = calculate_Fd_NPNIn(complete_diet_info['Fd_NPNCPIn'])
        complete_diet_info['Fd_NPNDMIn'] = calculate_Fd_NPNDMIn(complete_diet_info['Fd_NPNCPIn'])
        complete_diet_info['Fd_CPAIn'] = calculate_Fd_CPAIn(complete_diet_info['Fd_CPIn'], complete_diet_info['Fd_CPARU'])
        complete_diet_info['Fd_CPBIn'] = calculate_Fd_CPBIn(complete_diet_info['Fd_CPIn'], complete_diet_info['Fd_CPBRU'])
        complete_diet_info['Fd_CPBIn_For'] = calculate_Fd_CPBIn_For(complete_diet_info['Fd_CPIn'],
                                                                    complete_diet_info['Fd_CPBRU'],
                                                                    complete_diet_info['Fd_For'])
        complete_diet_info['Fd_CPBIn_Conc'] = calculate_Fd_CPBIn_Conc(complete_diet_info['Fd_CPIn'],
                                                                    complete_diet_info['Fd_CPBRU'],
                                                                    complete_diet_info['Fd_Conc'])
        complete_diet_info['Fd_CPCIn'] = calculate_Fd_CPCIn(complete_diet_info['Fd_CPIn'], complete_diet_info['Fd_CPCRU'])
        complete_diet_info['Fd_CPIn_ClfLiq'] = calculate_Fd_CPIn_ClfLiq(diet_info['Fd_Category'], 
                                                                        complete_diet_info['Fd_DMIn'], 
                                                                        diet_info['Fd_CP'])
        complete_diet_info['Fd_CPIn_ClfDry'] = calculate_Fd_CPIn_ClfDry(diet_info['Fd_Category'], 
                                                                        complete_diet_info['Fd_DMIn'], 
                                                                        diet_info['Fd_CP'])
        complete_diet_info['Fd_OMIn'] = calculate_Fd_OMIn(complete_diet_info['Fd_DMIn'], complete_diet_info['Fd_AshIn'])

        # Rumen Degraded and Undegraded Protein
        complete_diet_info['Fd_rdcRUPB'] = calculate_Fd_rdcRUPB(complete_diet_info['Fd_For'], 
                                                                complete_diet_info['Fd_Conc'],
                                                                complete_diet_info['Fd_KdRUP'],
                                                                coeff_dict)
        complete_diet_info['Fd_RUPBIn'] = calculate_Fd_RUPBIn(complete_diet_info['Fd_For'], 
                                                                complete_diet_info['Fd_Conc'],
                                                                complete_diet_info['Fd_KdRUP'],
                                                                complete_diet_info['Fd_CPBIn'],
                                                                coeff_dict)
        complete_diet_info['Fd_RUPIn'] = calculate_Fd_RUPIn(complete_diet_info['Fd_CPIn'], 
                                                            complete_diet_info['Fd_CPAIn'],
                                                            complete_diet_info['Fd_CPCIn'],
                                                            complete_diet_info['Fd_NPNCPIn'],
                                                            complete_diet_info['Fd_RUPBIn'],
                                                            coeff_dict)
        complete_diet_info['Fd_RUP_CP'] = calculate_Fd_RUP_CP(complete_diet_info['Fd_CPIn'], 
                                                            complete_diet_info['Fd_RUPIn'])
        complete_diet_info['Fd_RUP'] = calculate_Fd_RUP(complete_diet_info['Fd_CPIn'],
                                                        complete_diet_info['Fd_RUPIn'],
                                                        complete_diet_info['Fd_DMIn'])
        complete_diet_info['Fd_RDP'] = calculate_Fd_RDP(complete_diet_info['Fd_CPIn'],
                                                        complete_diet_info['Fd_CP'],
                                                        complete_diet_info['Fd_RUP'])
    
    if testmethod == 'new':
        complete_diet_info = complete_diet_info.assign(
            TT_dcFdNDF_Lg=lambda df: calculate_TT_dcFdNDF_Lg(df['Fd_NDF'], df['Fd_Lg']),
            Fd_DNDF48=lambda df: calculate_Fd_DNDF48(df['Fd_Conc'], df['Fd_DNDF48']),
            TT_dcFdNDF_48h=lambda df: calculate_TT_dcFdNDF_48h(df['Fd_DNDF48']),
            TT_dcFdNDF_Base=lambda df: calculate_TT_dcFdNDF_Base(Use_DNDF_IV, df['Fd_Conc'],
                                                                df['TT_dcFdNDF_Lg'], df['TT_dcFdNDF_48h']),
            Fd_DigNDFIn_Base=lambda df: calculate_Fd_DigNDFIn_Base(df['Fd_NDFIn'], df['TT_dcFdNDF_Base']),
            Fd_NPNCPIn=lambda df: calculate_Fd_NPNCPIn(df['Fd_CPIn'], df['Fd_NPN_CP']),
            Fd_NPNIn=lambda df: calculate_Fd_NPNIn(df['Fd_NPNCPIn']),
            Fd_NPNDMIn=lambda df: calculate_Fd_NPNDMIn(df['Fd_NPNCPIn']),
            Fd_CPAIn=lambda df: calculate_Fd_CPAIn(df['Fd_CPIn'], df['Fd_CPARU']),
            Fd_CPBIn=lambda df: calculate_Fd_CPBIn(df['Fd_CPIn'], df['Fd_CPBRU']),
            Fd_CPBIn_For=lambda df: calculate_Fd_CPBIn_For(df['Fd_CPIn'], df['Fd_CPBRU'], df['Fd_For']),
            Fd_CPBIn_Conc=lambda df: calculate_Fd_CPBIn_Conc(df['Fd_CPIn'], df['Fd_CPBRU'], df['Fd_Conc']),
            Fd_CPCIn=lambda df: calculate_Fd_CPCIn(df['Fd_CPIn'], df['Fd_CPCRU']),
            Fd_CPIn_ClfLiq=lambda df: calculate_Fd_CPIn_ClfLiq(df['Fd_Category'], df['Fd_DMIn'], df['Fd_CP']),
            Fd_CPIn_ClfDry=lambda df: calculate_Fd_CPIn_ClfDry(df['Fd_Category'], df['Fd_DMIn'], df['Fd_CP']),
            Fd_OMIn=lambda df: calculate_Fd_OMIn(df['Fd_DMIn'], df['Fd_AshIn']),
            Fd_rdcRUPB=lambda df: calculate_Fd_rdcRUPB(df['Fd_For'], df['Fd_Conc'], df['Fd_KdRUP'], coeff_dict),
            Fd_RUPBIn=lambda df: calculate_Fd_RUPBIn(df['Fd_For'], df['Fd_Conc'], df['Fd_KdRUP'], df['Fd_CPBIn'], coeff_dict),
            Fd_RUPIn=lambda df: calculate_Fd_RUPIn(df['Fd_CPIn'], df['Fd_CPAIn'], df['Fd_CPCIn'],
                                                df['Fd_NPNCPIn'], df['Fd_RUPBIn'], coeff_dict),
            Fd_RUP_CP=lambda df: calculate_Fd_RUP_CP(df['Fd_CPIn'], df['Fd_RUPIn']),
            Fd_RUP=lambda df: calculate_Fd_RUP(df['Fd_CPIn'], df['Fd_RUPIn'], df['Fd_DMIn']),
            Fd_RDP=lambda df: calculate_Fd_RDP(df['Fd_CPIn'], df['Fd_CP'], df['Fd_RUP'])
    )


    # FA Intakes
    column_names_FAIn = ['Fd_C120',
                         'Fd_C140',
                         'Fd_C160',
                         'Fd_C161',
                         'Fd_C180',
                         'Fd_C181t',
                         'Fd_C181c',
                         'Fd_C182',
                         'Fd_C183',
                         'Fd_OtherFA'
                         ]

    if testmethod == 'old':
        for col in column_names_FAIn:
            complete_diet_info[f"{col}In"] = complete_diet_info.apply(lambda row: row[f"{col}_FA"] / 100 * row['Fd_FA'] / 100 * row['Fd_DMIn'], axis=1)

    if testmethod == 'new':
        complete_diet_info = complete_diet_info.assign(
            **{f"{col}In": lambda df, col=col: df[f"{col}_FA"] / 100 * df['Fd_FA'] / 100 * df['Fd_DMIn'] for col in column_names_FAIn}
        )

    return complete_diet_info

In [34]:
def DI_run_NASEM(testmethod):
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    user_diet, animal_input, equation_selection = nd.read_csv_input('./input.csv')
        
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')
    feed_data.reset_index(inplace=True)
    feed_data = feed_data.rename(columns={"Fd_Name": "Feedstuff"})

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()
    diet_info_initial.insert(1, 'Fd_DMInp', Fd_DMInp.reindex(diet_info_initial['Feedstuff']).values)
    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    diet_info_initial['Fd_DNDF48'] = 0  

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del(list_of_feeds, Fd_DMInp)

    # Create infusion_input
    if equation_selection['use_infusions'] == 0:
         # If no infusions then all values are set to 0
        no_infusion_input = {'Inf_Acet_g': 0,
                             'Inf_ADF_g': 0,
                             'Inf_Arg_g': 0,
                             'Inf_Ash_g': 0,
                             'Inf_Butr_g': 0,
                             'Inf_CP_g': 0,
                             'Inf_CPARum_CP': 0,
                             'Inf_CPBRum_CP': 0,
                             'Inf_CPCRum_CP': 0,
                             'Inf_dcFA': 0,
                             'Inf_dcRUP': 0,
                             'Inf_DM_g': 0,
                             'Inf_EE_g': 0,
                             'Inf_FA_g': 0,
                             'Inf_Glc_g': 0,
                             'Inf_His_g': 0,
                             'Inf_Ile_g': 0,
                             'Inf_KdCPB': 0,
                             'Inf_Leu_g': 0,
                             'Inf_Lys_g': 0,
                             'Inf_Met_g': 0,
                             'Inf_NDF_g': 0,
                             'Inf_NPNCP_g': 0,
                             'Inf_Phe_g': 0,
                             'Inf_Prop_g': 0,
                             'Inf_St_g': 0,
                             'Inf_Thr_g': 0,
                             'Inf_Trp_g': 0,
                             'Inf_ttdcSt': 0,
                             'Inf_Val_g': 0,
                             'Inf_VFA_g': 0,
                             'Inf_Location': 0
                             }
    elif equation_selection['use_infusions'] == 1:
        infusion_input = nd.read_infusion_input('./infusion_input.csv')
    else:
        raise ValueError(f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")


########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8: 
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'], 
            animal_input['Trg_MilkProd'], 
            animal_input['An_BW'], 
            animal_input['An_BCS'],
            animal_input['An_LactDay'], 
            animal_input['Trg_MilkFatp'], 
            animal_input['Trg_MilkTPp'], 
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers    
    elif equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'], 
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF, 
            animal_input['An_BW'], 
            animal_input['An_BW_mature'], 
            animal_input['An_PrePartWk'], 
            nd.coeff_dict)

    
    elif equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'], 
            animal_input['An_BW'], 
            animal_input['An_PrePartWk'], 
            animal_input['An_GestDay'], 
            animal_input['An_GestLength'], 
            Dt_NDF, 
            nd.coeff_dict)
        
    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: "+ str(equation_selection['DMIn_eqn']))

    del(Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    # diet_info = nd.calculate_diet_info(animal_input['DMI'],
    #                                    animal_input['An_StatePhys'],
    #                                    equation_selection['Use_DNDF_IV'],
    #                                    diet_info=diet_info_initial,
    #                                    coeff_dict=nd.coeff_dict)
    diet_info = DIcalculate_diet_info(animal_input['DMI'],animal_input['An_StatePhys'], equation_selection['Use_DNDF_IV'], diet_info_initial, nd.coeff_dict, testmethod = testmethod)
    
    
    return diet_info

In [38]:
DI_run_NASEM(testmethod = 'old')


,Feedstuff,Fd_DMInp,Fd_Libr,UID,Fd_Index,Fd_Category,Fd_Type,Fd_DM,Fd_Conc,Fd_Locked,...,Fd_C120In,Fd_C140In,Fd_C160In,Fd_C161In,Fd_C180In,Fd_C181tIn,Fd_C181cIn,Fd_C182In,Fd_C183In,Fd_OtherFAIn
0,Alfalfa meal,0.334821,NRC 2020,NRC16F1,1,Plant Protein,Forage,90.749,0,1,...,0.001541,0.000771,0.026531,0.002252,0.004417,NaN,0.003986,0.027813,0.059311,0.005398
1,Canola meal,0.274554,NRC 2020,NRC16F28,28,Plant Protein,Concentrate,89.128,100,1,...,NaN,0.000318,0.016522,0.001532,0.003786,0.001038,0.076513,0.053231,0.013054,0.003256
2,"Corn silage, typical",0.223214,NRC 2020,NRC16F48,48,Grain Crop Forage,Forage,35.361,0,1,...,0.000395,0.000597,0.022951,0.000467,0.003121,0.000005,0.024762,0.061452,0.010614,0.004371
3,"Corn grain HM, coarse grind",0.167411,NRC 2020,NRC16F1072,256,Energy Source,Concentrate,72.267,100,1,...,NaN,0.003414,0.019363,0.000180,0.002911,NaN,0.035317,0.081658,0.002379,0.001369


In [39]:
DI_run_NASEM(testmethod = 'new')

,Feedstuff,Fd_DMInp,Fd_Libr,UID,Fd_Index,Fd_Category,Fd_Type,Fd_DM,Fd_Conc,Fd_Locked,...,Fd_C120In,Fd_C140In,Fd_C160In,Fd_C161In,Fd_C180In,Fd_C181tIn,Fd_C181cIn,Fd_C182In,Fd_C183In,Fd_OtherFAIn
0,Alfalfa meal,0.334821,NRC 2020,NRC16F1,1,Plant Protein,Forage,90.749,0,1,...,0.001541,0.000771,0.026531,0.002252,0.004417,NaN,0.003986,0.027813,0.059311,0.005398
1,Canola meal,0.274554,NRC 2020,NRC16F28,28,Plant Protein,Concentrate,89.128,100,1,...,NaN,0.000318,0.016522,0.001532,0.003786,0.001038,0.076513,0.053231,0.013054,0.003256
2,"Corn silage, typical",0.223214,NRC 2020,NRC16F48,48,Grain Crop Forage,Forage,35.361,0,1,...,0.000395,0.000597,0.022951,0.000467,0.003121,0.000005,0.024762,0.061452,0.010614,0.004371
3,"Corn grain HM, coarse grind",0.167411,NRC 2020,NRC16F1072,256,Energy Source,Concentrate,72.267,100,1,...,NaN,0.003414,0.019363,0.000180,0.002911,NaN,0.035317,0.081658,0.002379,0.001369


In [41]:
import timeit

# Assuming DI_run_NASEM is the function you want to test
def DI_run_NASEM_old():
    DI_run_NASEM(testmethod='old')

def DI_run_NASEM_new():
    DI_run_NASEM(testmethod='new')

# Run timeit to measure execution time for the 'old' method
time_old = timeit.timeit(DI_run_NASEM_old, number=200)

# Run timeit to measure execution time for the 'new' method
time_new = timeit.timeit(DI_run_NASEM_new, number=200)

# Print the results
print(f"DI_run_NASEM with 'old' method took {time_old:.6f} seconds.")
print(f"DI_run_NASEM with 'new' method took {time_new:.6f} seconds.")

DI_run_NASEM with 'old' method took 2.961133 seconds.
DI_run_NASEM with 'new' method took 2.349381 seconds.
